In [1]:
"""Pipeline for the project.

Here we design and define the pipeline for the project. We will play around with different
automated machine learning libraries and see which one works best for our project.

Structure:
    1. H2O.ai
    2. TPOT
    3. AutoML
    4. AutoKeras
"""

'Pipeline for the project.\n\nHere we design and define the pipeline for the project. We will play around with different\nautomated machine learning libraries and see which one works best for our project.\n\nStructure:\n    1. H2O.ai\n    2. TPOT\n    3. AutoML\n    4. AutoKeras\n'

# H2O.ai

In [5]:
# 1. Imports, Variables, Functions
# imports
import numpy as np, pandas as pd
import h2o, h5py
from h2o.automl import H2OAutoML

# variables
input_space = "B1"
data_path_X = "/aloy/web_checker/current/full/%s/%s/%s.001/sign4/sign4.h5" % (
    input_space[:1],
    input_space,
    input_space,
)
data_path_Y = "/aloy/web_checker/current/full/%s/%s/%s.001/sign0/sign0.h5" % (
    input_space[:1],
    input_space,
    input_space,
)
h2o.init(max_mem_size="16G", nthreads=8)

# functions


def get_X_indexes(data_path_X, data_path_Y):
    """Get X indexes for Y compounds!

    Args:
    - data_path_X: str
        Path to X data.
    - data_path_Y:str
        Path to Y data.
    Returns:
    - np.array
        Array of X indexes for Y data.
    """
    # load Y data
    with h5py.File(data_path_Y, "r") as f:
        Y = f["V"][:]
        keys_Y = f["keys"][:].astype("<U")

    # load X data
    with h5py.File(data_path_X, "r") as f:
        X = f["V"][:]
        keys_X = f["keys"][:].astype("<U")

    return np.where(np.isin(keys_X, keys_Y))[0]


# 2. Load Data

# get X indexes for Y compounds
x_indexes = get_X_indexes(data_path_X, data_path_Y)

# load X data
with h5py.File(data_path_X, "r") as f:
    X = f["V"][x_indexes][:]
    keys_X = f["keys"][x_indexes].astype("<U")

# load Y data
with h5py.File(data_path_Y, "r") as f:
    Y = f["V"][:]
    keys_Y = f["keys"][:].astype("<U")

assert (keys_X == keys_Y).all, "Keys do not match!"

# 3. Preprocess Data

# imports
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, roc_auc_score
import h2o
from h2o.frame import H2OFrame


# 4. Train Model
def train_model(X, Y, feature_index):
    """
    Trains H2OAutoML model for a specific feature (column) in Y.

    Args:
    - X: np.array
        The feature matrix.
    - Y: np.array
        The target matrix.
    - feature_index: int
        The column index in Y to train the model on.

    Returns:
    - The trained H2OAutoML model.
    """
    # Converting to H2OFrame
    hf = h2o.H2OFrame(np.hstack((X, Y[:, feature_index].reshape(-1, 1))))

    # Splitting into training and validation sets for early stopping
    train, valid = hf.split_frame(ratios=[0.8], seed=1234)

    # Setting target and predictors
    y = hf.columns[-1]
    x = hf.columns[:-1]

    # Training the model
    aml = H2OAutoML(
        max_runtime_secs=3600,
        seed=1,
        nfolds=5,
        balance_classes=True,
        keep_cross_validation_predictions=True,
    )
    aml.train(x=x, y=y, training_frame=train, validation_frame=valid)

    return aml


# 5. Evaluate Model
def evaluate_model(model, X_test, y_test):
    """
    Evaluates the performance of the trained model.

    Args:
    - model: H2OAutoML
        The trained H2OAutoML model.
    - X_test: np.array
        The test feature matrix.
    - y_test: np.array
        The test target array.

    Returns:
    - dict
        A dictionary containing evaluation metrics.
    """
    # Converting test data to H2OFrame
    test_hf = h2o.H2OFrame(np.hstack((X_test, y_test.reshape(-1, 1))))

    # Making predictions
    preds = model.predict(test_hf)
    predictions = h2o.as_list(preds["predict"]).values.flatten()

    # Calculating metrics
    acc = accuracy_score(y_test, predictions)
    auc = roc_auc_score(y_test, predictions)

    return {"accuracy": acc, "auc": auc}


# Create stratified folds and undersample
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
rus = RandomUnderSampler(random_state=1234)

# Training and evaluating models for each feature in Y
results = []
for feature_index in range(Y.shape[1]):
    print(f"Training model for feature {feature_index}...")
    for train_index, test_index in skf.split(X, Y[:, feature_index]):
        # Undersample the train set
        X_resampled, Y_resampled = rus.fit_resample(
            X[train_index], Y[train_index, feature_index]
        )

        # Train the model
        model = train_model(X_resampled, Y_resampled.reshape(-1, 1), feature_index)

        # Evaluate the model
        result = evaluate_model(model, X[test_index], Y[test_index, feature_index])
        results.append(result)

        # Optionally: Save each model
        # model_path = h2o.save_model(model=aml.leader, path=f"/path/to/save/model_feature_{feature_index}", force=True)

# 6. Save Model (if not saved above)
# Can be implemented based on specific requirements

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 min 37 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_ddalton_w72bha
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15.97 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Training model for feature 0...
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
15:46:57.846: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
15:46:57.871: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


15:46:58.403: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


15:46:58.555: _response param, We have detected that your response column has only 2 unique values (0/1). If you w

H2OJobCancelled: Job<$03017f00000132d4ffffffff$_8efb98176d68ee16a41c4d1c1fa147b3> was cancelled by the user.

# TPOT


In [7]:
# 1. Imports, Variables, Functions
# imports
import numpy as np
import pandas as pd
import h5py
from tpot import TPOTClassifier
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, roc_auc_score
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# variables
input_space = "B1"
data_path_X = "/aloy/web_checker/current/full/%s/%s/%s.001/sign4/sign4.h5" % (
    input_space[:1],
    input_space,
    input_space,
)
data_path_Y = "/aloy/web_checker/current/full/%s/%s/%s.001/sign0/sign0.h5" % (
    input_space[:1],
    input_space,
    input_space,
)


# functions
def get_X_indexes(data_path_X, data_path_Y):
    """Get X indexes for Y compounds!

    Args:
    - data_path_X: str
        Path to X data.
    - data_path_Y:str
        Path to Y data.
    Returns:
    - np.array
        Array of X indexes for Y data.
    """
    # load Y data
    with h5py.File(data_path_Y, "r") as f:
        Y = f["V"][:]
        keys_Y = f["keys"][:].astype("<U")

    # load X data
    with h5py.File(data_path_X, "r") as f:
        X = f["V"][:]
        keys_X = f["keys"][:].astype("<U")

    return np.where(np.isin(keys_X, keys_Y))[0]


# 2. Load Data

# get X indexes for Y compounds
x_indexes = get_X_indexes(data_path_X, data_path_Y)

# load X data
with h5py.File(data_path_X, "r") as f:
    X = f["V"][x_indexes][:]
    keys_X = f["keys"][x_indexes].astype("<U")

# load Y data
with h5py.File(data_path_Y, "r") as f:
    Y = f["V"][:]
    keys_Y = f["keys"][:].astype("<U")

assert (keys_X == keys_Y).all, "Keys do not match!"


# 3. Preprocess Data
# No changes needed here


# 4. Train Model
def train_model(X, Y, feature_index):
    """
    Trains TPOT model for a specific feature (column) in Y for binary classification.

    Args:
    - X: np.array
        The feature matrix.
    - Y: np.array
        The target matrix.
    - feature_index: int
        The column index in Y to train the model on.

    Returns:
    - The trained TPOT model.
    """
    # Convert data to pandas DataFrame for TPOT
    data_df = pd.DataFrame(X)
    data_df["target"] = Y[:, feature_index]

    # Create stratified folds and undersample
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
    rus = RandomUnderSampler(random_state=1234)

    # Initialize TPOTClassifier
    tpot = TPOTClassifier(
        generations=5,
        population_size=20,
        verbosity=2,
        scoring="roc_auc",
        cv=skf,
        random_state=42,
        n_jobs=-1,
    )

    # Training with undersampling in each fold
    for train_index, test_index in skf.split(
        data_df.drop("target", axis=1), data_df["target"]
    ):
        X_train, X_test = data_df.iloc[train_index, :-1], data_df.iloc[test_index, :-1]
        y_train, y_test = data_df.iloc[train_index, -1], data_df.iloc[test_index, -1]

        # Undersample the training set
        X_res, y_res = rus.fit_resample(X_train, y_train)

        # Fit TPOT with the resampled dataset
        tpot.fit(X_res, y_res)

    return tpot


# 5. Evaluate Model
def evaluate_model(model, X_test, y_test):
    """
    Evaluates the performance of the trained model.

    Args:
    - model: TPOTClassifier
        The trained TPOT model.
    - X_test: np.array
        The test feature matrix.
    - y_test: np.array
        The test target array.

    Returns:
    - dict
        A dictionary containing evaluation metrics.
    """
    predictions = model.predict(X_test)

    # Calculating metrics
    acc = accuracy_score(y_test, predictions)
    auc = roc_auc_score(y_test, predictions)

    return {"accuracy": acc, "auc": auc}


# Training and evaluating models for each feature in Y
results = []
for feature_index in range(Y.shape[1]):
    print(f"Training model for feature {feature_index}...")

    # Train the model
    model = train_model(X, Y, feature_index)

    # Evaluate the model
    # Assuming separate test data is available as X_test, y_test
    # result = evaluate_model(model, X_test, Y_test[:, feature_index])
    # results.append(result)

    # Optionally: Save each model
    # model.export('tpot_exported_pipeline_feature_{}.py'.format(feature_index))

# 6. Save Model (if not saved above)
# Can be implemented based on specific requirements

/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Training model for feature 0...


/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isin

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/tpot/builtins/__init__.py:36

...

                                                                              
Generation 5 - Current best internal CV score: 1.0
                                                                              
Best pipeline: GaussianNB(input_matrix)


/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isin

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/ddalton/miniconda3/envs/smol_p/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isin

RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [4]:
Y

<Closed HDF5 dataset>

In [ ]:
x_indexes

array([     37,     166,     641, ..., 1008896, 1009001, 1009089])

In [ ]:
import os

os.listdir("/aloy/web_checker/current/full/B/B1/B1.001/sign4/sign4.h5")

['models', 'stats', 'diags', '.STATUS', 'sign4.h5']

In [ ]:
len()

3972